# 현재 날짜 기준으로 다음날 예상 혼잡도 통계량 제공
## 5일 전 자료까지 존재
# 예고자료 -> 매일 17시에 업데이트

In [40]:
url = 'https://www.airport.kr/co/ko/cpr/statisticPredictCrowdedOfRoute.do'

In [41]:
#각종 패키지 설치

import pandas as pd
import numpy as np

!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

from urllib.request import urlopen
#from bs4 import BeautifulSoup
#from html_table_parser import parser_functions as parser
from selenium.webdriver.common.by import By

In [116]:
import datetime
now = datetime.datetime.now()
now.month

4

In [120]:
crowd_predict = pd.DataFrame()#columns = {'터미널','시간','일본_1','중국_1','동남아_1','미주_1','유럽_1','오세아니아_1','기타_1','일본_2','중국_2','동남아_2','미주_2','유럽_2','오세아니아_2','기타_2'})


# 터미널 선택창
xpath_terminal = '//*[@id="SEL_TM"]'

# 예고일 선택장
xpath_day = '//*[@id="RESVE_DE"]'

# 검색 버튼
xpath_search = '/html/body/section[2]/div/div/article[2]/div[1]/div[2]/div[2]/button'

# 내용 
xpath_content = '/html/body/section[2]/div/div/article[2]/table/tbody'

# 웹드라이버 열기
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")
driver.get(url)

for i in range(1,3) :
    if i == 2 :
        driver.find_element(by=By.XPATH, value = xpath_terminal).click() # 터미널 선택 클릭
        time.sleep(1)
        driver.find_element(by=By.XPATH, value = xpath_terminal).send_keys(Keys.DOWN) # 제 2터미널 선택 클릭
        driver.find_element(by=By.XPATH, value = xpath_terminal).send_keys(Keys.RETURN) # 제 2터미널 선택 클릭
    
    driver.find_element(by=By.XPATH, value = '//*[@id="RESVE_DE"]').text
    driver.find_element(by=By.XPATH, value = xpath_search).click() # 검색 버튼 클릭
    a = pd.DataFrame(driver.find_element(by=By.XPATH, value = xpath_content).text.split("\n"))
    a.columns = ['col']
    a = a['col'].str.split(" ", expand=True)
    a['터미널'] = '제{}여객터미널'.format(i)
    a.columns = ['시간','일본_{}'.format(i),'중국_{}'.format(i),'동남아_{}'.format(i),'미주_{}'.format(i),
                 '유럽_{}'.format(i),'오세아니아_{}'.format(i),'기타_{}'.format(i),'터미널']
    if i == 2 :
        a = a.drop(['시간'],axis=1)
    crowd_predict = pd.concat([crowd_predict,a], axis=i-1)
#crowd_predict = crowd_predict[['터미널','시간','일본','중국','동남아','미주','유럽','오세아니아','기타']]

C:\Users\USER\AppData\Local\Temp/ipykernel_2876/1381046086.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")


In [136]:
driver.find_element(by=By.XPATH, value = '//*[@id="RESVE_DE"]').text.replace(" ",'').replace("\n",'').split(")")

['2022.04.22(금',
 '2022.04.21(목',
 '2022.04.20(수',
 '2022.04.19(화',
 '2022.04.18(월',
 '2022.04.17(일',
 '2022.04.16(토',
 '']

In [78]:
crowd_predict = crowd_predict.drop(['터미널'], axis=1)[['시간','일본_1','일본_2','중국_1','중국_2','동남아_1','동남아_2',
                                    '미주_1','미주_2','유럽_1','유럽_2','오세아니아_1','오세아니아_2','기타_1','기타_2']]

In [79]:
['일본','중국','동남아','미주','유럽','오세아니아','기타']

for i in ['일본','중국','동남아','미주','유럽','오세아니아','기타'] :
    crowd_predict[i+"_통합"] = crowd_predict[i+'_1'].astype(int) + crowd_predict[i+'_2'].astype(int)

In [91]:
total = crowd_predict[['시간','일본_통합','중국_통합','동남아_통합','미주_통합','유럽_통합','오세아니아_통합','기타_통합']]

In [196]:
#total['합계'] = total.iloc[:,1:].sum(axis=1)

In [57]:
total.to_excel('공향_예상_혼잡도_0422.xlsx', index = False)

In [162]:
import pandas as pd
import glob

try:
    path = r'C:\Users\USER\Downloads\공항예상혼잡도/'
    files = glob.glob(path + "*.xls")
    excel = pd.DataFrame()
    for file_name in files:
        df = pd.read_excel(file_name)
        excel = excel.append(df, ignore_index=True)
    print(excel)

except Exception as ex:
    print('error' + str(ex))

        시간  일본  중국  동남아  미주   유럽  오세아니아   기타
0     00~1   0   0    0   0  181      0    0
1     01~2   0   0    0   0   76      0    0
2     02~3   0   0    0   0    1      0    0
3     03~4   0   0    0   0    0      0    0
4     04~5   4   0    0   0    0      0    0
..     ...  ..  ..  ...  ..  ...    ...  ...
163  19~20   0   0   79  73   15     47   36
164  20~21   0   0    2   0   53      6   61
165  21~22   0   0    0   0  149      0  147
166  22~23   0   0    0   0   79      0  131
167  23~24   0   0    0   0    1      0  193

[168 rows x 8 columns]


In [185]:
for i in pd.date_range('20220416', periods = 7) :
    print("{}년{}월{}일".format(i.year, i.month, i.day))
    ee['시간'] = ee['시간'].apply(lambda x : "{}년{}월{}일 ".format(str(i.year)[2:], i.month, i.day)+x if x =='00~1' else x)

2022년4월16일
2022년4월17일
2022년4월18일
2022년4월19일
2022년4월20일
2022년4월21일
2022년4월22일


In [188]:
ee.to_excel('공향_예상_혼잡도_0416~22.xlsx', index = False)